## Population Accessability
Determines the population/employment within a given time along a given network from a set of input facility locations.

Inputs: 
- Point data containing the locations of key services;
- Mesh block (polygon) data containing  field to add up (usually population)
- Cutoff time (the time in minutes that you want to investigate the accessability for)
- A network dataset

Outputs:
- An updated closest facility layer: Routes is updated to include the population of the mesh block it connects; Facilities is updated to contain the total population that is within the cutoff and closest to that facility
- A table containing the total population accessable to each facility
- A table containing the total population accessable to all facilities

This code has two components. One which finds the required routes using closest facility, and a second which outputs a table containiing the total population closest to each facility.

In [1]:
import arcpy
from arcgis import GeoAccessor
import pandas as pd

In [3]:
#paths for data to be used
MB16_CQ_pop = "MainDatasets\\MB16_CQ_pop"
MB16_CQ_empl = "MainDatasets\\MB16_CQ_empl"
MB_test = "MB_test"
city_centre =  "Test Runs\\CityCentre"

In [5]:
#population accessability - main code (run functions below first)
closest_facility = Closest_Facility_To_Centres(Incidents_Polygons = MB_test, Input_Facilities = city_centre)

Find_Total_Field(Incidents_Polygons=MB16_CQ_empl,
                 Facilities=closest_facility.name + "\\Facilities", 
                 Routes=closest_facility.name + "\\Routes", 
                 Field_to_sum = "MB_Employment_Count",
                 Total_Statistic="Total_Statistic", 
                 Routes_Statistics="Routes_Statistics")


(<Result 'C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\TestProject\\TestProject.gdb\\Routes_Statistics'>, <Result 'C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\TestProject\\TestProject.gdb\\Total_Statistic'>)

In [1]:
def Closest_Facility_To_Centres(Incidents_Polygons="MB16_CQ_empl", Network_Dataset="TransitNetwork_ND", Mode = "Public transit time", Cutoff=None, Input_Facilities="LR1_Stops", Output_Layer_Name = "Closest Facility"):  # Closest_Facility_To_Centres
    '''Runs the closest facility tool from a single function. 
    Inputs: A polygon layer for the incidents, a network dataset, a cutoff time in minutes, point input facilities, and a string name for 
    the output layer.
    Outputs: A closest facility network analysis layer
    Assumptions: A fixed start time at Thursday 11am.
    '''
    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Process: Make Closest Facility Analysis Layer (Make Closest Facility Analysis Layer) (na)
    Closest_Facility = arcpy.na.MakeClosestFacilityAnalysisLayer(network_data_source=Network_Dataset, layer_name=Output_Layer_Name, travel_mode=Mode, travel_direction="TO_FACILITIES", cutoff=Cutoff, number_of_facilities_to_find=1, time_of_day="4/01/1900 11:00:00 AM", time_zone="LOCAL_TIME_AT_LOCATIONS", time_of_day_usage="END_TIME", line_shape="ALONG_NETWORK", accumulate_attributes=[], generate_directions_on_solve="NO_DIRECTIONS", ignore_invalid_locations="SKIP")[0]

    # Process: Add Locations (Facilities) (Add Locations) (na)
    Closest_Facility_with_Facilities_ = arcpy.na.AddLocations(in_network_analysis_layer=Closest_Facility, sub_layer="Facilities", in_table=Input_Facilities, field_mappings="", search_tolerance="5000 Meters", sort_field="", search_criteria=[["StopConnectors", "SHAPE"], ["Streets", "SHAPE"], ["Stops", "SHAPE"], ["StopsOnStreets", "SHAPE"], ["TransitNetwork_ND_Junctions", "NONE"]], match_type="MATCH_TO_CLOSEST", append="APPEND", snap_to_position_along_network="NO_SNAP", snap_offset="5 Meters", exclude_restricted_elements="EXCLUDE", search_query=[])[0]

    # Process: Feature To Point (Feature To Point) (management)
    Incident_Points = "Incidents_points"
    arcpy.management.FeatureToPoint(in_features=Incidents_Polygons, out_feature_class=Incident_Points, point_location="CENTROID")  

    # Process: Add Locations (Incidents) (Add Locations) (na)
    Closest_Facility_with_Facilities_and_Incidents_ = arcpy.na.AddLocations(in_network_analysis_layer=Closest_Facility_with_Facilities_, sub_layer="Incidents", in_table=Incident_Points, field_mappings="", search_tolerance="5000 Meters", sort_field="", search_criteria=[["StopConnectors", "SHAPE"], ["Streets", "SHAPE"], ["Stops", "SHAPE"], ["StopsOnStreets", "SHAPE"], ["TransitNetwork_ND_Junctions", "NONE"]], match_type="MATCH_TO_CLOSEST", append="APPEND", snap_to_position_along_network="NO_SNAP", snap_offset="5 Meters", exclude_restricted_elements="EXCLUDE", search_query=[])[0]

    # Process: Solve (Solve) (na)
    Closest_Facility_solved, Solve_Succeeded = arcpy.na.Solve(in_network_analysis_layer=Closest_Facility_with_Facilities_and_Incidents_, ignore_invalids="SKIP", terminate_on_solve_error="TERMINATE", simplification_tolerance="", overrides="")
    
    return Closest_Facility_solved

In [2]:
#Join the field to the meshblock layer
def Find_Total_Field(Incidents_Polygons="MainDatasets\\MB16_CQ_empl", 
                     Facilities="Closest Facility\\Facilities", 
                     Routes="Closest Facility\\Routes", 
                     Field_to_sum = "MB_Employment_Count",           
                     Total_Statistic="C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\TestProject\\TestProject.gdb\\Total_Statistic", 
                     Routes_Statistics="C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\TestProject\\TestProject.gdb\\Routes_Statistics"):  # Find_Total_Field_Within_Distance
    '''Adds up a particular field in the incidents which have the same closest facility.
    To be run after closest facility.
    Inputs: The incident polygons used in the closest facility tool, containing the required field (e.g. population) to be added up;
            The routes and facilities layers output from closest facility
    Outputs:
        (Implicit) A point dataset of the incidents.
        (Implicit) An updated route layer containing the population count.
        (Implicit) A table containing the total population by facility.
        (Implicit) An updated facility layer containing the population count.
        (Implicit) A table containig the total population
        (Explicit) A tuple containing the two tables.'''
    
    Transfer_Fields=[Field_to_sum]
    Statistics_Field_s_=[[Field_to_sum, "SUM"]]
    SUM_Statistics_Field_s_=[["SUM_"+Field_to_sum, "SUM"]]
    SUM_Transfer_Fields=["SUM_"+Field_to_sum]
    
    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Process: Feature To Point (Feature To Point) (management)
    Incidents_Points = "Incidents_points"
    arcpy.management.FeatureToPoint(in_features=Incidents_Polygons, out_feature_class=Incidents_Points, point_location="CENTROID")

    # Process: Join Field (Join Field) (management)
    #adding a population field to the routes
    Routes_2_ = arcpy.management.JoinField(in_data=Routes, in_field="IncidentID", join_table=Incidents_Points, join_field="OBJECTID", fields=Transfer_Fields)[0]

    # Process: Summary Statistics (Summary Statistics) (analysis)
    #adding up all the population with routes to the same facility
    totals_by_facility = arcpy.analysis.Statistics(in_table=Routes_2_, out_table=Routes_Statistics, statistics_fields=Statistics_Field_s_, case_field=["FacilityID"])

    # Process: Join Field (2) (Join Field) (management)
    Updated_Facilities_with_Population = arcpy.management.JoinField(in_data=Facilities, in_field="ObjectID", join_table=Routes_Statistics, join_field="FacilityID", fields=SUM_Transfer_Fields)[0]

    # Process: Summary Statistics (2) (Summary Statistics) (analysis)
    overall_total = arcpy.analysis.Statistics(in_table=Updated_Facilities_with_Population, out_table=Total_Statistic, statistics_fields=SUM_Statistics_Field_s_, case_field=[])
    
    return totals_by_facility, overall_total

In [ ]:
def add_source_information(OD_Layer = "OD Cost Matrix", Origin_Source, Destination_Source):
    '''Takes an Origin Destination analysis layer, and adds the origin and destination source layer information to the output fields
    '''
    OD_df = GeoAccessor.from_featureclass(OD_Layer + "\")
    
    origins_df = GeoAccessor.from_featureclass(feature_layer_name)